In [1]:
import numpy as np
import pandas as pd
import pyodbc
import csv
import pyworms

# DEBUG
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from warnings import filterwarnings
filterwarnings("ignore", category=UserWarning, message='.*')

# Output file path
outdir = "D:\\00-GCOOS\\00-MBON\\CAGES\\FL\\data\\"

# THE ORIGINAL CAGES DATABASE FILE is served via WAF but cannot be queried from there
# -> download to local HD
#db_file = 'https://gcoos4.geos.tamu.edu/WAF/MBON/CAGES/CAGES.accdb'
db_file = "D:\\00-GCOOS\\00-MBON\\CAGES_ORIG\CAGES.accdb"
user = 'user'
password = 'pw'
# open the database connection:
cnxn = pyodbc.connect('DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={};Uid={};Pwd={};'.format(db_file, user, password))

### GET DATA TABLES

# Get joined Stations and Physical tables
# --
query = 'SELECT st.*, ph.[Start Depth], ph.[End Depth], ph.[Distance Towed], ph.[Soak Time] FROM "Florida Stations" AS st LEFT JOIN "Florida Physical" AS ph ON \
st.[Station Code] = ph.[Station Code]'
st_ph = pd.read_sql(query, cnxn)
# Soak time got converted to datetime with year and all... Source just had hours:minutes, with hours as zero. 
# -> First drop lines where soak time not given 
st_ph.drop(st_ph[st_ph['Soak Time'].isna()].index, inplace=True)
# -> Get just minutes
st_ph['TowTime'] = st_ph['Soak Time'].dt.minute.astype('int')
# Drop Soak Time 
st_ph.drop(columns=['Soak Time'], inplace=True)

# Get Hydrological data
# --
query = 'SELECT * FROM "Florida Hydrological"'
hydro = pd.read_sql(query, cnxn)
# Get lowest depth (surface) and deepest depth (bottom) values.
# In source data they are in order from smallest to biggest depth -> use first, last
hydro_surface = hydro.groupby('Station Code').agg({'Depth': 'first', 'Temperature': 'first', 'Conductivity': 'first', 'pH': 'first', 'Salinity': 'first', 'DO': 'first'}).reset_index()
hydro_bottom = hydro.groupby('Station Code').agg({'Depth': 'last', 'Temperature': 'last', 'Conductivity': 'last', 'pH': 'last', 'Salinity': 'last', 'DO': 'last'}).reset_index()
# Rename lowest depth/surface (Depth1) and deepest depth/bottom (Depth2) columns for merging
for acol in hydro_surface.columns:
    if 'Station' in acol:
        continue
    else:
        hydro_surface = hydro_surface.rename(columns={acol: acol + '1'})
for acol in hydro_bottom.columns:
    if 'Station' in acol:
        continue
    else:
        hydro_bottom = hydro_bottom.rename(columns={acol: acol + '2'})

# MERGE surface and bottom values ...
merged_hydro = pd.merge(hydro_surface, hydro_bottom, on=['Station Code'])

# Merge the hydro data to the stations + physical data
st_ph = pd.merge(st_ph, merged_hydro, on=['Station Code'],how='left')




In [6]:
st_ph.rename(columns={'Bay code': 'Bay Code'}, inplace=True)

In [7]:
st_ph

,Station Code,Reference Code,Bay Code,YYYY,MM,DD,Latitude,Longitude,Start Depth,End Depth,Distance Towed,TowTime,Depth1,Temperature1,Conductivity1,pH1,Salinity1,DO1,Depth2,Temperature2,Conductivity2,pH2,Salinity2,DO2
0,1,APM2004051101,AP,2004,5,11,29.682067,-85.221983,4.4,3.6,0.20,10,0.2,26.6,27.9,7.9,17.1,6.8,3.6,26.6,50.4,8.1,33.1,5.4
1,2,APM2002070701,AP,2002,7,7,29.682267,-85.220817,4.4,4.8,0.20,10,0.2,28.8,55.8,8.0,37.1,5.6,4.6,28.8,56.0,8.0,37.2,5.5
2,3,APM2001100208,AP,2001,10,2,29.683100,-85.219967,4.6,4.6,0.19,10,0.2,25.2,52.0,8.0,34.2,6.2,3.0,25.1,52.0,8.0,34.3,6.3
3,4,APM2004091005,AP,2004,9,10,29.684650,-85.219583,3.1,3.1,0.19,10,0.2,26.8,20.0,8.2,11.8,9.1,3.1,26.8,21.9,8.2,13.2,8.9
4,5,APM2004100805,AP,2004,10,8,29.684867,-85.219117,2.5,3.1,0.20,10,0.2,24.9,31.0,8.1,19.3,8.2,3.4,24.5,43.9,8.1,28.3,6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7323,7325,CHM1991102115,CH,1991,10,21,26.951333,-82.109167,0.9,1.6,0.22,10,0.2,24.9,32.4,NaN,20.4,5.5,0.9,24.9,32.6,NaN,20.2,6.5
7324,7326,CHM1991102110,CH,1991,10,21,26.948833,-82.106000,1.3,1.6,0.21,10,0.6,25.0,32.4,NaN,20.9,6.4,0.6,25.0,32.4,NaN,20.9,6.4
7325,7327,CHM1991102111,CH,1991,10,21,26.948833,-82.106000,1.5,1.5,0.22,10,0.7,25.0,32.4,NaN,20.9,6.4,0.7,25.0,32.4,NaN,20.9,6.4
7326,7328,CHM1991102112,CH,1991,10,21,26.948833,-82.106000,1.5,1.9,0.23,10,0.7,25.0,32.4,NaN,20.9,6.4,0.7,25.0,32.4,NaN,20.9,6.4


In [3]:
# Get joined CPUE and Species tables
query = 'SELECT cp.*, sp.[Scientific Name], sp.[Common Name] FROM "Florida CPUE" AS cp LEFT JOIN "Florida Species" AS sp ON \
cp.[Species Code] = sp.[Species Code]'
cpue_specs = pd.read_sql(query, cnxn)

# Convert ScientificName from NoneType to string:
cpue_specs.loc[:, 'ScientificName'] = cpue_specs['Scientific Name'].astype('str')
# Drop the rows where the scientific name not known (Missing in the source files)
cpue_specs.drop(cpue_specs[cpue_specs['ScientificName'] == 'None'].index, inplace=True)
# drop 'Scientific Name'
cpue_specs.drop('Scientific Name', axis=1, inplace=True)

In [4]:
cpue_specs

,Bay Code,Station Code,YYYY,MM,DD,Species Code,cpue,Common Name,ScientificName
0,TB,4745,1989,4,4,8.858020e+09,11.184,Blackcheek tonguefish,Symphurus plagiusa
1,TB,4746,1989,4,4,6.177010e+09,16.776,Pink shrimp,Farfantepenaeus (Penaeus) duorarum
2,TB,4746,1989,4,4,6.189010e+09,33.552,Blue crab,Callinectes sapidus
3,TB,4746,1989,4,4,8.747020e+09,1207.872,Bay anchovy,Anchoa mitchilli
4,TB,4746,1989,4,4,8.777180e+09,5.592,hardhead sea catfish,Ariopsis felis
...,...,...,...,...,...,...,...,...,...
191341,CH,7327,1991,10,21,6.177010e+09,0.000,Brown shrimp,Farfantepenaeus (Penaeus) aztecus
191342,CH,7328,1991,10,21,8.858030e+09,0.000,Lined sole,Achirus lineatus
191343,CH,7328,1991,10,21,8.835440e+09,0.000,Silver perch,Bairdiella chrysoura
191345,CH,7328,1991,10,21,6.177010e+09,0.000,Brown shrimp,Farfantepenaeus (Penaeus) aztecus


In [8]:

# merge cpue_specs and stations + physical + hydrological
cpue_specs = pd.merge(cpue_specs, st_ph, on=['Station Code','YYYY','MM','DD','Bay Code'],how='left')





In [9]:

cpue_specs

# !!! JATKA!
# merge more tables from access db!


,Bay Code,Station Code,YYYY,MM,DD,Species Code,cpue,Common Name,ScientificName,Reference Code,Latitude,Longitude,Start Depth,End Depth,Distance Towed,TowTime,Depth1,Temperature1,Conductivity1,pH1,Salinity1,DO1,Depth2,Temperature2,Conductivity2,pH2,Salinity2,DO2
0,TB,4745,1989,4,4,8.858020e+09,11.184,Blackcheek tonguefish,Symphurus plagiusa,TBM1989040412,27.798000,-82.477667,6.7,NaN,0.20,10.0,0.2,24.0,43.6,8.1,28.0,8.6,6.7,23.4,43.7,8.1,28.1,8.1
1,TB,4746,1989,4,4,6.177010e+09,16.776,Pink shrimp,Farfantepenaeus (Penaeus) duorarum,TBM1989040413,27.798000,-82.477667,7.7,NaN,0.20,10.0,0.2,24.0,43.6,8.1,28.0,8.6,7.7,23.4,43.7,8.1,28.1,8.1
2,TB,4746,1989,4,4,6.189010e+09,33.552,Blue crab,Callinectes sapidus,TBM1989040413,27.798000,-82.477667,7.7,NaN,0.20,10.0,0.2,24.0,43.6,8.1,28.0,8.6,7.7,23.4,43.7,8.1,28.1,8.1
3,TB,4746,1989,4,4,8.747020e+09,1207.872,Bay anchovy,Anchoa mitchilli,TBM1989040413,27.798000,-82.477667,7.7,NaN,0.20,10.0,0.2,24.0,43.6,8.1,28.0,8.6,7.7,23.4,43.7,8.1,28.1,8.1
4,TB,4746,1989,4,4,8.777180e+09,5.592,hardhead sea catfish,Ariopsis felis,TBM1989040413,27.798000,-82.477667,7.7,NaN,0.20,10.0,0.2,24.0,43.6,8.1,28.0,8.6,7.7,23.4,43.7,8.1,28.1,8.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183220,CH,7327,1991,10,21,6.177010e+09,0.000,Brown shrimp,Farfantepenaeus (Penaeus) aztecus,CHM1991102111,26.948833,-82.106000,1.5,1.5,0.22,10.0,0.7,25.0,32.4,NaN,20.9,6.4,0.7,25.0,32.4,NaN,20.9,6.4
183221,CH,7328,1991,10,21,8.858030e+09,0.000,Lined sole,Achirus lineatus,CHM1991102112,26.948833,-82.106000,1.5,1.9,0.23,10.0,0.7,25.0,32.4,NaN,20.9,6.4,0.7,25.0,32.4,NaN,20.9,6.4
183222,CH,7328,1991,10,21,8.835440e+09,0.000,Silver perch,Bairdiella chrysoura,CHM1991102112,26.948833,-82.106000,1.5,1.9,0.23,10.0,0.7,25.0,32.4,NaN,20.9,6.4,0.7,25.0,32.4,NaN,20.9,6.4
183223,CH,7328,1991,10,21,6.177010e+09,0.000,Brown shrimp,Farfantepenaeus (Penaeus) aztecus,CHM1991102112,26.948833,-82.106000,1.5,1.9,0.23,10.0,0.7,25.0,32.4,NaN,20.9,6.4,0.7,25.0,32.4,NaN,20.9,6.4


In [ ]:


# -- CPUE and Stations
query = 'SELECT * FROM "Alabama CPUE" AS cpue LEFT JOIN "Alabama Stations" AS stations ON \
cpue.[Station Code] = stations.[Station Code]'
cpuedata = pd.read_sql(query, cnxn)
# Remove duplicate columns
cpuedata = cpuedata.loc[:, ~cpuedata.columns.duplicated(keep='first')]


# -- TRAWLS and SPECIES
query = 'SELECT * FROM "Alabama Trawls" AS trawls LEFT JOIN "Alabama Species" AS species ON \
trawls.[Species Code] = species.[Species Code]'
trawlsdata = pd.read_sql(query, cnxn)

# Remove duplicate columns
trawlsdata = trawlsdata.loc[:, ~trawlsdata.columns.duplicated(keep='first')]

# Convert Species Code back to integer
intcols = ['Species Code']
for acol in intcols:
    #trawlsdata[acol] = trawlsdata[acol].astype('Int64')
    trawlsdata.loc[:, acol] = trawlsdata[acol].astype('Int64')
    cpuedata.loc[:, acol] = cpuedata[acol].astype('Int64')
# Drop outdated original data columns (up to date values will be extracted at DarwinCore alignment stage via the WoRMS API)
dropcols = ['Phylum','Class','Family','Common Name','Prior Name']
for acol in dropcols:
    trawlsdata.drop(acol, axis=1, inplace=True)

# Merge CPUE and Trawls data
df = pd.merge(cpuedata, trawlsdata, on=['Sample Code','Species Code'],how='left')

# -- Gear
query = 'SELECT * FROM "Alabama Gear"'
df = pd.merge(df, pd.read_sql(query, cnxn), on=['Gear Code'],how='left')

# -- Hydrological
query = 'SELECT * FROM "Alabama Hydrological"'
df = pd.merge(df, pd.read_sql(query, cnxn), on=['Sample Code'],how='left')

# -- Lengths
query = 'SELECT * FROM "Alabama Lengths"'
lengths = pd.read_sql(query, cnxn)
#df = pd.merge(df, pd.read_sql(query, cnxn), on=['Sample Code','Species Code'],how='left')
intcols = ['Species Code']
for acol in intcols:
    lengths.loc[:, acol] = lengths[acol].astype('Int64')

df = pd.merge(df, lengths, on=['Sample Code','Species Code'],how='left')

# Close the database connection
cnxn.close()

# Generate Datetime field
df['datestr'] = df['YYYY'].astype(str) + '-' + df['MM'].astype(str) + '-' + df['DD'].astype(str) + ' 12:00'
df['Datetime'] = pd.to_datetime(df['datestr'], utc=True)

# drop columns that are not needed for output
df.drop(['datestr', 'YYYY', 'MM', 'DD'], axis=1, inplace=True)

# sort by date
df.sort_values(['Datetime'], axis=0, ascending=True, inplace=True, ignore_index=True)
# Format datetime string
df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%dT%H:%MZ')

# Get rid of spaces in column names
for acol in df.columns.to_list():
    df.rename({acol: acol.replace(" ","")}, axis="columns", inplace=True)

# Convert ScientificName from NoneType to string:
df.loc[:, 'ScientificName'] = df['ScientificName'].astype('str')
# Drop the rows where the scientific name not known (Missing in the source files)
df.drop(df[df['ScientificName'] == 'None'].index, inplace=True)

### OUTPUT TO FILE
cols_out = ['Datetime','Latitude', 'Longitude','SampleCode', 'StationCode','Station','Description','WaterBody','Salinity','Temperature','DO','GearCode','Gear','SpeciesCode','ScientificName','cpue','Measured','TotalNumber','TotalWeight','Commercial','Length']
# Write merged data out to a .csv file
df[cols_out].to_csv(outdir + "CAGES_CPUE_AL.csv", encoding='utf-8', index=False)

In [ ]:
### Generate ERDDAP datasets .XML -snippet using templates according to variable type

## Paths 
# ouput
xml_output = outdir + "CAGES_AL_XML.txt"
# template files
xml_header_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\" + "cages_header_template_AL.txt"
xml_int_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\" + "int_xml_template.txt"
xml_float_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\" + "float_xml_template.txt"
xml_string_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\" + "string_xml_template.txt"

# get types
dt = df.dtypes.to_dict()
int_cols = []
float_cols = []
string_cols = []

# list types
for akey in dt.keys():
     if 'nt64' in str(dt[akey]):
        int_cols.append(akey)
     elif 'float' in str(dt[akey]):
         float_cols.append(akey)
     elif 'obj' in str(dt[akey]) or 'str' in str(dt[akey]):
         string_cols.append(akey)

# These belong to the header snippet (no need to generate separately):
header_vars = ['Datetime', 'Latitude', 'Longitude']

# START concatenating the tamplate snippets together
# --

# 1st, write the header to the output file
with open(xml_output, "w") as output_file:
    with open(xml_header_temp, "r") as file:
        output_file.write(file.read())

# Loop throgh the rest
for acol in cols_out:
    isPhysicalMeasurement = False
    if acol in header_vars:
        continue
    elif acol in int_cols:
        template_file = xml_int_temp
    elif acol in float_cols:
        template_file = xml_float_temp
        isPhysicalMeasurement = True
    else:
        template_file = xml_string_temp
    
    # Open the xml template file for reading    
    with open(template_file, 'r') as tempfile:
        # Read the contents of the file
        contents = tempfile.read()
        # Modify the contents as needed
        mod_contents = contents.replace('_VARNAME_', acol)
        
        # Try to figure out the units for some
        if isPhysicalMeasurement:
            if 'temperature' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'degree_Celcius')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Temperature')
                if 'air' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'air_temperature')
                elif 'surface' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_surface_temperature')
                elif 'bottom' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_temperature_at_sea_floor')
                else:
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_temperature')
            elif 'turbidity' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'NTU')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Turbidity')
                mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_turbidity')
            elif 'salinity' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'PSU')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Salinity')
                if 'surface' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_surface_salinity')
                elif 'bottom' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_salinity_at_sea_floor')
                else:
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_salinity')
            elif 'DO' in acol or 'dissolved' in acol.lower() or 'oxygen' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'mg l-1')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Dissolved O2')
                mod_contents = mod_contents.replace('_DESTNAME_', 'mass_concentration_of_oxygen_in_sea_water')
            else:
                mod_contents = mod_contents.replace('_DESTNAME_',acol)

    # Open the ouput file for appending
    with open(xml_output, 'a') as file2:
        # Write the modified contents back to the file
        file2.write(mod_contents)

### Add the closing tag to the output xml file:
with open(xml_output, 'a') as output_file:
    output_file.write("</dataset>")